External Location registration

In [0]:
from pyspark.sql.types import *

# Optional: define schema explicitly (recommended for prod)
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("department", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("created_at", DateType(), True)
])

autoLoaderStream = (
    spark.readStream
        .format("cloudFiles")
        .schema(schema)  # remove if using schema inference
        .option("cloudFiles.format", "csv")
        .option("header", "true")

        # 🔴 THIS ENABLES MANAGED FILE EVENTS
        .option("cloudFiles.useManagedFileEvents", "true")

        .option("cloudFiles.schemaLocation", "/Volumes/main/raw/_schemas/employees")
        .load("/Volumes/main/raw/employees/")
)

(
    autoLoaderStream.writeStream
        .format("delta")
        .option("checkpointLocation", "/Volumes/main/raw/_checkpoints/employees")
        .trigger(availableNow=True)
        .toTable("main.raw.employees_bronze")
)